In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization
from isic_helper import compute_pauc, compute_auc

In [2]:
def blend_optimizer(oof_preds_df, oof_columns, folds, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 10.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights

In [3]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_ensemble"
fold_column = "fold"

boosting_model_names = ["xgb", "xgb", "lgb"]
boosting_versions = ["v1", "v2", "v6"]
boosting_modes = ["train", "train", "train"]
boosting_oof_columns = [f"oof_{model_name}_{version}" for model_name, version in zip(boosting_model_names, boosting_versions)]

cnn_model_names = ["efficientnet_b2", "efficientnet_b2", "resnet18"]
cnn_versions = ["v1", "v2", "v1"]
cnn_modes = ["pretrain", "pretrain", "pretrain"]
cnn_oof_columns = [f"oof_{model_name}_{version}" for model_name, version in zip(cnn_model_names, cnn_versions)]

model_names = boosting_model_names + cnn_model_names
versions = boosting_versions + cnn_versions
modes = boosting_modes + cnn_modes
oof_columns = boosting_oof_columns + cnn_oof_columns

paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    mode = modes[idx]
    model_identifier = f"{model_name}_{version}"
    oof_preds_model_df = pd.read_csv(f"{path}/oof_preds_{model_identifier}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}

all_folds = np.unique(oof_preds_df["fold"])
weights = blend_optimizer(
    oof_preds_df, boosting_oof_columns + cnn_oof_columns, all_folds,
    init_points=50, n_iter=100
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |    w4     |    w5     |
-------------------------------------------------------------------------------------------------
| 1         | 0.1768    | 0.09359   | 4.991     | 1.134     | 0.4997    | 6.854     | 4.87      |
| 2         | 0.1794    | 8.977     | 6.475     | 8.97      | 7.211     | 8.314     | 8.276     |
| 3         | 0.1794    | 8.336     | 9.57      | 3.68      | 4.948     | 3.395     | 6.194     |
| 4         | 0.1793    | 9.775     | 0.9643    | 7.442     | 2.925     | 2.987     | 7.525     |
| 5         | 0.1785    | 0.1866    | 5.237     | 8.644     | 3.888     | 2.122     | 4.752     |
| 6         | 0.1791    | 5.647     | 3.494     | 9.759     | 0.3782    | 7.943     | 3.579     |
| 7         | 0.1793    | 7.48      | 9.145     | 3.727     | 9.649     | 0.8139    | 0.4245    |
| 8         | 0.1786    | 2.968     | 3.637     | 4.903     | 6.685     | 6.734     | 5.721     |
| 9         | 0.1783

In [4]:
weights

[9.184200253001404,
 0.8552034877824392,
 2.4021046742664947,
 4.764737619598357,
 3.2308662644235215,
 1.282657759225282]

In [5]:
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")

Model: xgb_v1 | Weightage: 9.184200253001404
Val AUC scores:
{'fold_1': 0.9763855695822203,
 'fold_2': 0.9572885288935904,
 'fold_3': 0.9635023430382749,
 'fold_4': 0.9709627048208559,
 'fold_5': 0.9687397106157167}
Val PAUC scores:
{'fold_1': 0.18367412688887158,
 'fold_2': 0.16537818384974395,
 'fold_3': 0.17194383259911888,
 'fold_4': 0.17426448565474031,
 'fold_5': 0.17318846840205077}
CV AUC OOF: 0.9464921630675764
CV PAUC OOF: 0.15296119492851015
CV AUC AVG: 0.9673757713901316
CV PAUC AVG: 0.17368981947890508
CV AUC STD: 0.006521637731286478
CV PAUC STD: 0.00587196296972231


Model: xgb_v2 | Weightage: 0.8552034877824392
Val AUC scores:
{'fold_1': 0.9751129113471103,
 'fold_2': 0.963817176741778,
 'fold_3': 0.9639184585241667,
 'fold_4': 0.9684925309805195,
 'fold_5': 0.9623259344160846}
Val PAUC scores:
{'fold_1': 0.18166898925849218,
 'fold_2': 0.17040633221405424,
 'fold_3': 0.17186716127341475,
 'fold_4': 0.17183237448677788,
 'fold_5': 0.16723127288063078}
CV AUC OOF: 0.9584

In [6]:
oof_preds_df.pivot_table(index="target", values=oof_columns + [ensemble_column], aggfunc="mean")

,oof_efficientnet_b2_v1,oof_efficientnet_b2_v2,oof_ensemble,oof_lgb_v6,oof_resnet18_v1,oof_xgb_v1,oof_xgb_v2
target,,,,,,,
0,0.000892,0.002586,10.850158,0.011006,0.000609,0.014516,0.009843
1,0.098317,0.178503,20.914759,0.386260,0.083599,0.328668,0.339775


In [7]:
oof_preds_df[oof_columns + [ensemble_column]].corr()

,oof_xgb_v1,oof_xgb_v2,oof_lgb_v6,oof_efficientnet_b2_v1,oof_efficientnet_b2_v2,oof_resnet18_v1,oof_ensemble
oof_xgb_v1,1.000000,0.875487,0.866551,0.434795,0.474942,0.397363,0.339156
oof_xgb_v2,0.875487,1.000000,0.914023,0.467436,0.513979,0.442805,0.325418
oof_lgb_v6,0.866551,0.914023,1.000000,0.462899,0.522323,0.423650,0.350965
oof_efficientnet_b2_v1,0.434795,0.467436,0.462899,1.000000,0.688298,0.689840,0.115906
oof_efficientnet_b2_v2,0.474942,0.513979,0.522323,0.688298,1.000000,0.629166,0.162520
oof_resnet18_v1,0.397363,0.442805,0.423650,0.689840,0.629166,1.000000,0.084207
oof_ensemble,0.339156,0.325418,0.350965,0.115906,0.162520,0.084207,1.000000
